# Notebook title

<a href="https://colab.research.google.com/github/harmslab/topiary/blob/main/alignment_to_ancestors_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Initialize environment

In [1]:
### THIS CELL SETS UP TOPIARY IN A GOOGLE COLAB ENVIRONMENT. 
### IF RUNNING LOCALLY, IT MAY BE SAFELY DELETED

#@title Install software (colab-only)

#@markdown Install the software by pressing the _Play_ button on the left.
#@markdown Please be patient. This will take several minutes. After the 
#@markdown installation is complete, <font color='blue'>the kernel will reboot and colab will
#@markdown complain that the kernel has died. This is normal.</font>

#@markdown If you wish to install raxml or generax, select the check boxes below. 
#@markdown Installing these packages takes a few minutes. Note: you can select
#@markdown the checkboxes and re-run the cell after doing the 
#@markdown initial installation. 

install_raxml = False #@param {type:"boolean"}
install_generax = False #@param {type:"boolean"}

# This assumes python 3.8 (the current colab default, 2022/12/07)
# If colab updates from python 3.8, change:

# 1. Miniconda3-py38_4.12.0-Linux-x86_64.sh
# 2. conda install --channel defaults conda python=3.8 --yes
# 3. Refs to /usr/local/lib/python3.8/site-packages

# Change these in both the "Install" and "Initialize" cells

from tqdm.auto import tqdm
import sys
import subprocess
import time
import os
import re

def run_install_cmd(bash_to_run,description):
    """
    Run an installation command.

    bash_to_run : str
        bash command as a string
    description : str
        description of what is being done
    """

    no_space = re.sub(" ","_",description)
    status_file = f"/content/software/{no_space}.installed"

    if os.path.isfile(status_file):
        print(f"{description} already installed.")
        return

    os.chdir("software")

    print(f"Installing {description}... ",end="",flush=True)
    f = open(f"{no_space}_tmp-script.sh","w")
    f.write(bash_to_run)
    f.close()

    result = subprocess.run(["bash",f"{no_space}_tmp-script.sh"],
                                                    stdout=subprocess.PIPE,
                                                    stderr=subprocess.PIPE,
                                                    text=True)
    
    if result.returncode != 0:
        print(result.stdout,flush=True)
        print(result.stderr,flush=True)
        raise RuntimeError("Installation failed!")

    
    f = open(f"{no_space}_stdout.txt","w")
    f.write(result.stdout)
    f.close()

    f = open(f"{no_space}_stderr.txt","w")
    f.write(result.stderr)
    f.close()

    os.chdir("..")

    f = open(status_file,'w')
    f.write("Installed\n")
    f.close()
    
    print("Complete.",flush=True)

    
miniconda = \
"""
unset PYTHONPATH
MINICONDA_INSTALLER_SCRIPT=Miniconda3-py38_4.12.0-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.anaconda.com/miniconda/$MINICONDA_INSTALLER_SCRIPT --quiet
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX
conda install --channel defaults conda python=3.8 --yes
conda update --channel defaults --all --yes
"""

conda_packages = \
"""
conda config --add channels conda-forge
conda config --add channels bioconda
conda install --channel defaults numpy pandas xlrd openpyxl matplotlib "muscle>=5.0" blast --yes --strict-channel-priority
"""

pip_packages = \
"""
# Install ghostscript binary (so toyplot doesn't complain)
wget https://github.com/ArtifexSoftware/ghostpdl-downloads/releases/download/gs1000/ghostscript-10.0.0-linux-x86_64.tgz
tar -zxf ghostscript-10.0.0-linux-x86_64.tgz
mv ghostscript-10.0.0-linux-x86_64/gs-1000-linux-x86_64 /usr/local/bin/gs

/usr/bin/python3 -m pip install mpi4py opentree ete3 dendropy biopython pastml toytree toyplot 
"""

raxml = \
"""
wget https://github.com/amkozlov/raxml-ng/releases/download/1.1.0/raxml-ng_v1.1.0_linux_x86_64.zip
unzip raxml-ng_v1.1.0_linux_x86_64.zip
cp raxml-ng /usr/local/bin
"""

generax = \
"""
apt-get install flex bison libgmp3-dev
rm -rf GeneRax
git clone --recursive https://github.com/BenoitMorel/GeneRax
cd GeneRax
./install.sh
cp build/bin/generax /usr/local/bin
cd ..
"""

topiary = \
"""
rm -rf topiary
git clone --branch main https://github.com/harmsm/topiary.git

cd topiary

# add --allow-run-as-root to mpirun calls
for x in `echo "./topiary/generax/_generax.py ./topiary/generax/_reconcile_bootstrap.py ./topiary/_private/mpi/mpi.py"`; do
    sed -i 's/\[\"mpirun\",/\[\"mpirun\",\"--allow-run-as-root\",/g' ${x}
done

/usr/bin/python3 -m pip install . -vv
cd ..
"""

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:

    description_list = ["miniconda","conda packages","pip packages"]
                    
    cmd_list = [miniconda,conda_packages,pip_packages]

    if install_raxml:
        description_list.append("raxml-ng")
        cmd_list.append(raxml)

    if install_generax:
        description_list.append("generax")
        cmd_list.append(generax)

    description_list.append("topiary")
    cmd_list.append(topiary)

    print("Setting up environment.",flush=True)

    # Make software directory (if not already there)
    os.system("mkdir -p software")

    # Add conda path to the current python session
    to_append = "/usr/local/lib/python3.8/site-packages"
    if to_append not in sys.path:
        sys.path.append(to_append)

    # Make sure that any new python session that spools up during the installations
    # has the correct site packages. 
    %env PYTHONSTARTUP=/content/software/python_startup.py
    f = open("/content/software/python_startup.py","w")
    f.write("import sys\n")
    f.write("sys.path.append('/usr/local/lib/python3.8/site-packages')\n")
    f.close()


    # Install each package
    pbar = tqdm(range(len(cmd_list)))
    for i in pbar:
        run_install_cmd(cmd_list[i],description_list[i])

        # Update status bar
        pbar.refresh()
        time.sleep(0.5)
        
    # This sleep step makes sure things are done writing to the display before
    # reset
    time.sleep(2)
    os._exit(0)


In [2]:
import topiary
import numpy as np
import pandas as pd 

### EVERYTHING AFTER THIS LINE IS IS USED TO SET UP TOPIARY IN A GOOGLE
### COLAB ENVIRONMENT. IF RUNNING LOCALLY, THE LINES BELOW MAY BE SAFELY
### DELETED. 

#@title Initialize environment

#@markdown  Run this cell to initialize the environment after installation.
#@markdown (This cell can also be run if the kernel dies during a calculation,
#@markdown allowing you to reload modules without having to
#@markdown reinstall). 

try:
    import google.colab
    RUNNING_IN_COLAB = True
except ImportError:
    RUNNING_IN_COLAB = False
except Exception as e: 
    err = "Could not figure out if runnning in a colab notebook\n"
    raise Exception(err) from e

if RUNNING_IN_COLAB:
    
    %env PYTHONPATH=""
    %env PYTHONSTARTUP=/content/software/python_startup.py
    %env TOPIARY_MAX_SLOTS=1

    topiary._in_notebook = "colab"

    from tqdm.auto import tqdm
    import sys
    import subprocess
    import time
    import os

    to_append = '/usr/local/lib/python3.8/site-packages'
    if to_append not in sys.path:
        sys.path.append(to_append)

    os.chdir("/content/")


## Tests

This should be deleted when using this as a template notebook. These should run on both colab and a local computer. 

### Generate an alignment from a seed dataframe

In [3]:
df_location = "https://raw.githubusercontent.com/harmslab/topiary/main/tests/data/seed-dataframes/good-seed-df.csv"
df = pd.read_csv(df_location)
df

,species,accession,name,aliases,sequence
0,Homo sapiens,Q9Y6Y9,LY96,ESOP1;Myeloid Differentiation Protein-2;LY-96;...,MLPFLFFSTLFSSIFTEAQKQYWVCNSSDASISYTYCDKMQYPISI...
1,Mus musculus,Q9JHF9,LY96,ESOP1;Myeloid Differentiation Protein-2;LY-96;...,MLPFILFSTLLSPILTESEKQQWFCNSSDAIISYSYCDHLKFPISI...
2,Gallus gallus,A0A1D5NZX9,LY96,ESOP1;Myeloid Differentiation Protein-2;LY-96;...,MFEFVFFILFTPGVSGFFCTSSDLELSYTFCDSSAHYFKLNMTPCS...
3,Danio rerio,A0A140LG20,LY96,ESOP1;Myeloid Differentiation Protein-2;LY-96;...,MALWCPSAFLCFIALSCMASEKAERKSLCSSEQVNFWYTFEGPLHY...
4,Homo sapiens,O95711,LY86,Lymphocyte Antigen 86;MD-1;LY86;RP105-associat...,MKGFTATLFLWTLIFPSCSGGGGGKAWPTHVVCSDSGLEVLYQSCD...
5,Mus musculus,O88188,LY86,Lymphocyte Antigen 86;MD-1;LY86;RP105-associat...,MNGVAAALLVWILTSPSSSDHGSENGWPKHTACNSGGLEVVYQSCD...
6,Danio rerio,E7F6I2,LY86,Lymphocyte Antigen 86;MD-1;LY86;RP105-associat...,MKTYFNMLLFLILGLVQMDRAHSQDPQWPLHTICNSNKLTVTYRSC...
7,Gallus gallus,Q90890,LY86,Lymphocyte Antigen 86;MD-1;LY86;RP105-associat...,MKTLNVLALVLVLLCINASTEWPTHTVCKEENLEIYYKSCDPQQDF...


In [4]:
topiary.seed_to_alignment(df_location)

----------------------------------------------------------------------
Checking blastp
----------------------------------------------------------------------

    installed:       Y
    binary_path:     /Users/harmsm/local/bin/blastp
    binary runs:     Y
    version:         2.13.0+
    minimum version: 2.0
    passes:          Y

----------------------------------------------------------------------
Checking makeblastdb
----------------------------------------------------------------------

    installed:       Y
    binary_path:     /Users/harmsm/local/bin/makeblastdb
    binary runs:     Y
    version:         2.13.0+
    minimum version: 2.0
    passes:          Y

----------------------------------------------------------------------
Checking muscle
----------------------------------------------------------------------

    installed:       Y
    binary_path:     /Users/harmsm/local/bin/muscle
    binary runs:     Y
    version:         5.1.osxarm64
    minimum version: 5.0
    

Process Process-1:
Traceback (most recent call last):
  File "/Users/harmsm/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/harmsm/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/harmsm/work/programming/git-clones/topiary/topiary/_private/animation.py", line 51, in _iterate
    time.sleep(self._delay)
KeyboardInterrupt


KeyboardInterrupt: 

### Get ancestors given an alignment

In [7]:
tiny_df_location = "https://raw.githubusercontent.com/harmslab/topiary/main/tests/data/tiny-phylo/initial-input/dataframe.csv"

topiary.alignment_to_ancestors(tiny_df_location,out_dir="ali-to-anc")

----------------------------------------------------------------------
Checking raxml-ng
----------------------------------------------------------------------

    installed:       Y
    binary_path:     /Users/harmsm/local/bin/raxml-ng
    binary runs:     Y
    version:         1.1.0
    minimum version: 1.1
    passes:          Y

----------------------------------------------------------------------
Checking generax
----------------------------------------------------------------------

    installed:       Y
    binary_path:     /Users/harmsm/local/bin/generax
    binary runs:     Y
    version:         2.0.4
    minimum version: 2.0
    passes:          Y

----------------------------------------------------------------------
Checking mpirun
----------------------------------------------------------------------

    installed:       Y
    binary_path:     /Users/harmsm/miniconda3/bin/mpirun
    binary runs:     Y
    version:         4.1.2
    minimum version: 0.0
    passes:   

  0%|          | 0/360 [00:00<?, ?it/s]


Top 10 models:

               model           AICc prob
               rtREV               0.843
                 PMB               0.057
                 DEN               0.047
            Blosum62               0.018
                 WAG               0.014
                  LG               0.010
                  VT               0.008
           JTT-DCMut               0.001
                 JTT               0.001
               DCMut               0.000


topiary ran a find_best_model calculation in ./00_find-model:

+ Completed in 0:00:06.548197 (H:M:S)
+ Wrote results to ./00_find-model/output

----------------------------------------------------------------------



----------------------------------------------------------------------

topiary is starting a ml_tree calculation in ./01_ml-tree:

Launching raxml-ng, 0:00:00.001679 (H:M:S)
Running '/Users/harmsm/local/bin/raxml-ng --search --msa alignment.phy --model rtREV --seed 385509525 --threads auto{10}'


topiary ran a

libc++abi: terminating with uncaught exception of type LibpllException: Could not load open newick file result/results/reconcile/geneTree.newick
libc++abi: terminating with uncaught exception of type LibpllException: Could not load open newick file result/results/reconcile/geneTree.newick
--------------------------------------------------------------------------
Primary job  terminated normally, but 1 process returned
a non-zero exit code. Per user-direction, the job has been aborted.
--------------------------------------------------------------------------


GeneRax was called as follow:
/Users/harmsm/local/bin/generax --families control.txt --species-tree species_tree.newick --prefix result --rec-model UndatedDL --seed 385509525 

General information:
- Output prefix: result
- Families information: control.txt
- Species tree: species_tree.newick
- You are running GeneRax without MPI (no parallelization)
- Random seed: 385509525
- Reconciliation model: UndatedDL
- DTL rates: global rates
- Infer ML reconciliation: ON
- Unrooted reconciliation likelihood: OFF
- Prune species tree mode: OFF

Gene tree correction information:
- Gene tree strategy: SPR
- Max gene SPR radius: 5

[00:00:00] Filtering invalid families...

End of instance initialization
[00:00:00] Starting species tree initialization...
[00:00:00] End of species tree initialization
[00:00:00] Filtering invalid families based on the starting species tree...

[00:00:00] Gathering statistics about the families...
[00:00:00] Input data information:
- Number of gene families: 1
- Numbe

libc++abi: terminating with uncaught exception of type LibpllException: Could not load open newick file result/results/reconcile/geneTree.newick
libc++abi: terminating with uncaught exception of type LibpllException: Could not load open newick file result/results/reconcile/geneTree.newick
Assertion failed: (geneTreeStrings.size() == 1), function optimizeGeneTreesSlave, file GeneRaxSlave.cpp, line 58.
sh: line 1:  5904 Abort trap: 6           /Users/harmsm/local/bin/generax optimizeGeneTrees result/results/reconcile/geneTree.newick mapping.link alignment.phy result/species_trees/inferred_species_tree.newick rtREV result/gene_optimization_2/dtl_rates.txt 0 0 0 1 0 0.000001 NONE 0 -1 1 1 1 3 result/results/reconcile/geneTree.newick result/results/reconcile/stats.txt 0 > result/gene_optimization_2/per_job_logs/reconcile_out.txt
--------------------------------------------------------------------------
mpirun noticed that process rank 2 with PID 0 on node banks exited on signal 6 (Abort tra



topiary ran a reconcile calculation in ./02_reconciliation:

+ Crashed after 0:00:04.726683 (H:M:S)
+ Please check ./02_reconciliation/working

----------------------------------------------------------------------




RuntimeError: 

In [6]:
topiary.pipeline.bootstrap_reconcile("ali-to-anc",2,overwrite=True)

FileNotFoundError: 
previous_run_dir 'ali-to-anc' does not exist



### Look at ancestral reconstruction output

In [ ]:
topiary.draw.tree("ali-to-anc/05_reconcile-bootstraps")

In [ ]:
anc_fasta = "ali-to-anc/03_ancestors/output/reconciled-tree_ancestors/ancestors.fasta"
with open(anc_fasta as f):
    for line in f:
        print(f,end="")

In [ ]:
df = pd.read_csv("ali-to-anc/03_ancestors/output/reconciled-tree_ancestors/ancestor-data.csv")
df[df.anc == "anc1"]

In [ ]:
topiary.alignment_to_ancestors("seed_to_alignment_vetsAEDotS/05_clean-aligned-dataframe.csv")